In [1]:
import warnings
import os
from tqdm import tqdm
from typing import List
import pandas as pd
from unidecode import unidecode
import plotly.express as px
import unicodedata
import math
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")


warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
# Hiển thị 6 chữ số thập phân
# pd.set_option('display.float_format', '{:.6f}'.format)

In [2]:
df = pd.read_parquet('data/preprocessed_videos.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70996 entries, 0 to 70995
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   CategoryType                     70996 non-null  object        
 1   author.downloadSetting           70996 non-null  object        
 2   author.duetSetting               70996 non-null  object        
 3   author.id                        70996 non-null  object        
 4   author.nickname                  70996 non-null  object        
 5   author.openFavorite              70996 non-null  bool          
 6   author.secUid                    70996 non-null  object        
 7   author.signature                 70996 non-null  object        
 8   author.stitchSetting             70996 non-null  object        
 9   author.uniqueId                  70996 non-null  object        
 10  author.verified                  70996 non-null  bool     

## tiền xử lý time và 0 followers


In [3]:
df['createTime'] = df['createTime'].dt.tz_localize(
    'UTC').dt.tz_convert("Asia/Ho_Chi_Minh")
df['collectTime'] = df['collectTime'].dt.tz_localize(
    'UTC').dt.tz_convert("Asia/Ho_Chi_Minh")

In [4]:
df = df.merge(
    df.groupby("author.uniqueId")["authorStats.followerCount"].max().reset_index().rename(
        columns={"authorStats.followerCount": "max_followerCount"}
    ),
    on="author.uniqueId",
    how="left"
)

df["authorStats.followerCount"] = df["max_followerCount"]
df.drop(columns=["max_followerCount"], inplace=True)

## Old score


In [5]:
def compute_old_weekly_score(df):
    """
    Tính điểm xếp hạng video theo tuần dựa trên:
    - Lượt xem tuyệt đối
    - Các chỉ số tương tác (like, share, comment)
    - Engagement rate

    Trả về dataframe đã thêm: old_score, old_rank
    """

    df_copy = df.copy()

    # Nếu chưa có cột tuần/năm thì tính lại từ thời gian
    if 'year' not in df_copy.columns or 'week' not in df_copy.columns:
        df_copy['year'] = df_copy['createTime'].dt.isocalendar().year
        df_copy['week'] = df_copy['createTime'].dt.isocalendar().week

    # Tính các chỉ số cần thiết
    df_copy['engagement_rate'] = (
        df_copy['statsV2.diggCount'] +
        df_copy['statsV2.commentCount'] +
        df_copy['statsV2.shareCount'] +
        df_copy['statsV2.collectCount']
    ) / (df_copy['statsV2.playCount'] + 1e-6)
    df_copy.loc[df_copy['statsV2.playCount'] < 1, 'engagement_rate'] = 0

    # Trọng số cho old_score
    weights = {
        'statsV2.playCount': 0.40,
        'engagement_rate': 0.10,
        'statsV2.shareCount': 0.15,
        'statsV2.commentCount': 0.10,
        'statsV2.diggCount': 0.25
    }

    def compute_score(group, df_copy):
        # Chuẩn hóa từng chỉ số
        for col in weights:
            max_val = group[col].max()
            group[f'norm_{col}'] = group[col] / (max_val if max_val > 0 else 1)

        # Tính điểm tổng hợp
        group[df_copy] = sum(
            weights[col] * group[f'norm_{col}'] for col in weights
        ) * 100

        # Xếp hạng
        group = group.sort_values(
            df_copy, ascending=False).reset_index(drop=True)
        group[f'{df_copy}_rank'] = np.arange(1, len(group) + 1)

        return group

    df_ranked = df_copy.groupby(
        ['year', 'week'], group_keys=False).apply(lambda group: compute_score(group, 'old_score'))

    return df_ranked

## New score


In [6]:
def compute_new_weekly_score(df):
    """
    Tính điểm xếp hạng video theo tuần dựa trên:
    - Lượt xem tuyệt đối
    - Views per 1000 followers
    - Engagement rate

    Trả về dataframe đã thêm: new_score, new_rank (theo từng tuần)
    """

    df_copy = df.copy()

    # Nếu chưa có cột tuần/năm thì tính lại từ thời gian
    if 'year' not in df_copy.columns or 'week' not in df_copy.columns:
        df_copy['year'] = df_copy['createTime'].dt.isocalendar().year
        df_copy['week'] = df_copy['createTime'].dt.isocalendar().week

    # Tính các chỉ số cần thiết
    df_copy['engagement_rate'] = (
        df_copy['statsV2.diggCount'] +
        df_copy['statsV2.commentCount'] +
        df_copy['statsV2.shareCount'] +
        df_copy['statsV2.collectCount']
    ) / (df_copy['statsV2.playCount'] + 1e-6)
    df_copy.loc[df_copy['statsV2.playCount'] < 1, 'engagement_rate'] = 0

    df_copy['views_per_followers'] = df_copy['statsV2.playCount'] / \
        (df_copy['authorStats.followerCount'] + 1e-6)

    # 2. Lấy followerCount cao nhất của mỗi user
    max_followers = (
        df_copy.groupby("author.uniqueId")["authorStats.followerCount"]
        .max()
        .reset_index()
        .rename(columns={"authorStats.followerCount": "max_followerCount"})
    )

    # 3. Merge lại vào df_copy
    df_copy = df_copy.merge(max_followers, on="author.uniqueId", how="left")

    # 4. Tính views_per_followers với follower > 0
    df_copy["views_per_followers"] = 0
    mask = df_copy["max_followerCount"] > 0
    df_copy.loc[mask, "views_per_followers"] = (
        df_copy.loc[mask, "statsV2.playCount"] /
        df_copy.loc[mask, "max_followerCount"]
    )

    # Trọng số cho old_score
    weights = {
        'statsV2.playCount': 0.40,
        'views_per_followers': 0.15,
        'engagement_rate': 0.10,
    }

    def compute_score(group, col_name):
        # Chuẩn hóa từng chỉ số
        for col in weights:
            max_val = group[col].max()
            group[f'norm_{col}'] = group[col] / (max_val if max_val > 0 else 1)

        # Tính điểm tổng hợp
        group[col_name] = sum(
            weights[col] * group[f'norm_{col}'] for col in weights
        ) * 100

        # Xếp hạng
        group = group.sort_values(
            col_name, ascending=False).reset_index(drop=True)
        group[f'{col_name}_rank'] = np.arange(1, len(group) + 1)

        return group

    df_ranked = df_copy.groupby(
        ['year', 'week'], group_keys=False).apply(lambda group: compute_score(group, 'new_score'))

    return df_ranked

In [7]:
# def get_topN(df, score_col, col_suffix, topN=50):
#     top_df = (df
#               .sort_values(['year', 'week', score_col], ascending=[True, True, False])
#               .groupby(['year', 'week'])
#               .head(topN)
#               [['year', 'week', 'video.id']]  # chỉ cần ID video
#               )
#     top_df = top_df.rename(columns={'id': f'id_{col_suffix}'})
#     return top_df


# # Lấy top 20 theo điểm mới và cũ
# top20_new = get_topN(df_ranked, 'new_score', 'new')
# # top20_old = get_topN(df_ranked, 'weekly_score', 'old')

# # # Bước 2: Gộp lại theo tuần
# # merged = pd.merge(
# #     top20_new,
# #     top20_old,
# #     how='outer',
# #     on=['year', 'week']
# # )

# # Bước 3: So sánh tập ID theo tuần
# results = []

# for (y, w), group in merged.groupby(['year', 'week']):
#     new_ids = set(group['id_new'].dropna())
#     old_ids = set(group['id_old'].dropna())
#     unique_to_new = new_ids - old_ids
#     results.append({
#         'year': y,
#         'week': w,
#         'num_new_only': len(unique_to_new),
#         'pct_changed': len(unique_to_new) / 20 * 100
#     })

# df_diff = pd.DataFrame(results).sort_values(['year', 'week'])

# # Tổng kết
# total_unique_to_new = df_diff['num_new_only'].sum()
# avg_diff_per_week = df_diff['num_new_only'].mean()

# print(
#     f"\n📊 Tổng số video chỉ có mặt trong top 20 của new_score (không có trong weekly_score): {total_unique_to_new}")
# print(
#     f"🔁 Trung bình thay đổi mỗi tuần: {avg_diff_per_week:.2f} video (~{avg_diff_per_week/20*100:.1f}%)")

# # Vẽ biểu đồ nếu cần

# # plt.figure(figsize=(10, 4))
# # plt.plot(df_diff['week'], df_diff['num_new_only'], marker='o')
# # plt.title("Số video khác biệt trong top 20 (new_score vs weekly_score) theo tuần")
# # plt.xlabel("Tuần")
# # plt.ylabel("Số video chỉ có trong top 20 (new_score)")
# # plt.grid(True, linestyle='--', alpha=0.6)
# # plt.tight_layout()
# # plt.show()

In [8]:
df_score = df.copy()

df_score['engagement_rate'] = (
    df_score['statsV2.diggCount'] +
    df_score['statsV2.commentCount'] +
    df_score['statsV2.shareCount'] +
    df_score['statsV2.collectCount']
) / (df_score['statsV2.playCount'] + 1e-6)

df_score['views_per_followers'] = df_score['statsV2.playCount'] / \
    (df_score['authorStats.followerCount'] + 1e-6)
# 2. Lấy followerCount cao nhất của mỗi user
max_followers = (
    df_score.groupby("author.uniqueId")["authorStats.followerCount"]
    .max()
    .reset_index()
    .rename(columns={"authorStats.followerCount": "max_followerCount"})
)

# 3. Merge lại vào df_score
df_score = df_score.merge(max_followers, on="author.uniqueId", how="left")

# 4. Tính views_per_followers với follower > 0
df_score["views_per_followers"] = 0
mask = df_score["max_followerCount"] > 0
df_score.loc[mask, "views_per_followers"] = (
    df_score.loc[mask, "statsV2.playCount"] /
    df_score.loc[mask, "max_followerCount"]
)
df_score.loc[df_score['statsV2.playCount'] < 1, 'engagement_rate'] = 0

In [9]:
columns_of_interest = [
    "author.uniqueId", "video.id",
    # "statsV2.diggCount", "statsV2.commentCount",
    # "statsV2.shareCount", "statsV2.collectCount",
    "authorStats.followerCount",
    # "authorStats.heartCount", "authorStats.videoCount",
    "statsV2.playCount", "views_per_followers", "engagement_rate"
]

In [10]:
px.scatter(df_score, x='statsV2.playCount', y='authorStats.followerCount')

In [11]:
df_score.loc[df_score['statsV2.playCount'].idxmax(), columns_of_interest]

author.uniqueId                  lethanhtuan4999
video.id                     7430429241394711826
authorStats.followerCount               127800.0
statsV2.playCount                     36400000.0
views_per_followers                   284.820031
engagement_rate                         0.034912
Name: 49291, dtype: object

In [12]:
# Lấy các cột cần thiết
cols = [
    "statsV2.playCount",
    "views_per_followers",
    "engagement_rate"
]

# Loại bỏ các hàng có playCount = 0 để tránh gây nhiễu
df_filtered = df_score[df_score["isAd"] != 0][cols]

# Tính hệ số tương quan Pearson
correlation_matrix = df_filtered.corr(method="pearson")

correlation_matrix

,statsV2.playCount,views_per_followers,engagement_rate
statsV2.playCount,1.000000,0.528855,-0.123074
views_per_followers,0.528855,1.000000,-0.116512
engagement_rate,-0.123074,-0.116512,1.000000


In [13]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# # Lọc dữ liệu: chỉ lấy video không quảng cáo và playCount > 0
# df_pca = df_score[(df["statsV2.playCount"] > 0)].copy()

# # Chọn các biến đầu vào PCA
# features = ["statsV2.playCount", "views_per_1000_followers", "engagement_rate"]

# # Chuẩn hoá dữ liệu
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df_pca[features])

# # Áp dụng PCA
# pca = PCA(n_components=3)
# pca.fit(X_scaled)

# # Lấy thành phần chính và trọng số (loading)
# pca_components = pd.DataFrame(
#     pca.components_,
#     columns=features,
#     index=[f"PC{i+1}" for i in range(3)]
# )

# # Lấy phần trăm phương sai giải thích
# explained_variance = pca.explained_variance_ratio_

# pca_components["ExplainedVariance"] = list(
#     explained_variance) + [None] * (3 - len(explained_variance))
# pca_components

In [14]:
# Tính lại các thành phần PCA để lấy giá trị điểm cho từng video
pca_scores = pca.transform(X_scaled)

# Tạo dataframe từ PCA scores
df_scores = pd.DataFrame(
    pca_scores,
    columns=["PC1", "PC2", "PC3"]
)

# Gắn lại tỷ lệ phương sai giải thích
explained_variance = pca.explained_variance_ratio_

# Tính điểm PCA tổng hợp có trọng số theo phương sai giải thích
df_scores["pca_score_weighted"] = (
    df_scores["PC1"] * explained_variance[0] +
    df_scores["PC2"] * explained_variance[1] +
    df_scores["PC3"] * explained_variance[2]
)

# Gắn lại với dataframe gốc (đã lọc)
df_pca_with_score = df_pca.reset_index(drop=True).copy()
df_pca_with_score["pca_score_weighted"] = df_scores["pca_score_weighted"]
df_pca_with_score[["pca_score_weighted"]+columns_of_interest
                  ].sort_values("pca_score_weighted", ascending=False)

NameError: name 'pca' is not defined

In [ ]:
df["authorStats.followerCount"].min()

## So sánh video ad vs. not ad


In [ ]:
ad_df = df_score[df_score['isAd'] == 1][columns_of_interest]
not_ad_df = df_score[df_score['isAd'] == 0][columns_of_interest]
px.scatter(ad_df, x='statsV2.playCount', y='engagement_rate')

In [ ]:
ad_df.describe()

In [ ]:
not_ad_df.describe()

In [ ]:
df_score.loc[df_score['views_per_followers'].idxmax(), columns_of_interest]